
```
{"intents": [
  {"tag": "greeting",
    "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up", "Hey", "greetings"],
    "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],
    "context_set": ""
  },
  {"tag": "goodbye",
    "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "cao", "see ya"],
    "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],
    "context_set": ""
  },
  {"tag": "stocks",
    "patterns": ["what stocks do I own?", "how are my shares?", "what companies am I investing in?", "what am I doing in the markets?"],
    "responses": ["You own the following shares: ABBV, AAPL, FB, NVDA and an ETF of the S&P 500 Index!"],
    "context_set": ""
  }
]
}
```


In [2]:
import random
import json
import pickle
import numpy as np
import os

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')

intents = json.loads(open("intents.json").read())
print(intents)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up', 'Hey', 'greetings'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day', 'bye', 'cao', 'see ya'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'stocks', 'patterns': ['what stocks do I own?', 'how are my shares?', 'what companies am I investing in?', 'what am I doing in the markets?'], 'responses': ['You own the following shares: ABBV, AAPL, FB, NVDA and an ETF of the S&P 500 Index!'], 'context_set': ''}]}


In [3]:
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        documents.append((word, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

print(words)
print(classes)
print(documents)

['a', 'am', 'anyone', 'are', 'bye', 'cao', 'company', 'cya', 'day', 'do', 'doing', 'good', 'goodbye', 'greeting', 'have', 'hello', 'hey', 'hi', 'how', 'i', 'in', 'investing', 'is', 'later', 'leaving', 'market', 'my', 'own', 'see', 'share', 'stock', 'the', 'there', 'up', 'what', 'whats', 'ya', 'you']
['goodbye', 'greeting', 'stocks']
[(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Whats', 'up'], 'greeting'), (['Hey'], 'greeting'), (['greetings'], 'greeting'), (['cya'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['bye'], 'goodbye'), (['cao'], 'goodbye'), (['see', 'ya'], 'goodbye'), (['what', 'stocks', 'do', 'I', 'own', '?'], 'stocks'), (['how', 'are', 'my', 'shares', '?'], 'stocks'), (['what', 'companies', 'am', 'I', 'investing', 'in', '?'], 'stocks'), (['what'

In [4]:
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 1, 0]], [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print(train_x[0])
print(train_y[0])



[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[0, 0, 1]


In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(train_y[0]), activation='softmax'))

model.compile(loss='mse', optimizer="sgd", metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
4/4 [==============================] - 1s 3ms/step - loss: 0.2185 - accuracy: 0.2533
Epoch 2/200
4/4 [==============================] - 0s 3ms/step - loss: 0.2137 - accuracy: 0.4200
Epoch 3/200
4/4 [==============================] - 0s 3ms/step - loss: 0.2108 - accuracy: 0.7400
Epoch 4/200
4/4 [==============================] - 0s 4ms/step - loss: 0.2078 - accuracy: 0.7800
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 0.2013 - accuracy: 0.7467
Epoch 6/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1999 - accuracy: 0.7667
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1915 - accuracy: 0.8667
Epoch 8/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1945 - accuracy: 0.7467
Epoch 9/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1831 - accuracy: 0.7800
Epoch 10/200
4/4 [==============================] - 0s 3ms/step - loss: 0.1779 - accuracy: 0.7933
Epoch 11/200
4/4 [===========

In [10]:
# in case we do not want to repeat training, we can save the results
model.save("intents.h5", hist)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [11]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('intents.h5')

In [14]:
def predict(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

  #["Hey", "how" ] = [0,0,0,1,0,0,0,1 ...]
  vec = [0] * len(words)
  for s in sentence_words:
      for i, word in enumerate(words):
          if word == s:
              vec[i] = 1

  res = model.predict(np.array([vec]))[0]
  # res = [0.23, 0.17, 0.6]
  ERROR_THRESHOLD = 0.1
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
      return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
  return return_list

def request(message):
  ints = predict(message)

  #print(ints)

  tag = ints[0]['intent']
  list_of_intents = intents['intents']
  for i in list_of_intents:
        if i['tag']  == tag:
            result = random.choice(i['responses'])
            break
  return result

done = False;
while not done:
    message = input("Enter a message: ")
    if message == "STOP":
        done = True
    else:
        print(request(message))

Enter a message: Hi
[{'intent': 'greeting', 'probability': '0.39708117'}, {'intent': 'goodbye', 'probability': '0.32971433'}, {'intent': 'stocks', 'probability': '0.27320457'}]
Good to see you again!
Enter a message: Bye
[{'intent': 'greeting', 'probability': '0.3821858'}, {'intent': 'goodbye', 'probability': '0.34658545'}, {'intent': 'stocks', 'probability': '0.27122876'}]
Good to see you again!
Enter a message: Good Bye
[{'intent': 'greeting', 'probability': '0.40445054'}, {'intent': 'goodbye', 'probability': '0.30565876'}, {'intent': 'stocks', 'probability': '0.2898907'}]
Hi there, how can I help?
Enter a message: STOP
